### 1. 简单的示例 

![image](docs/stream_warn.png)

In [ ]:
# 1.1 简单的示例

from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # pretrained YOLO11n model

# Run batched inference on a list of images
results = model([   "https://ultralytics.com/images/bus.jpg", 
                    "https://ultralytics.com/images/zidane.jpg"], stream=True)  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk

### 2. 多种输入源

![image](docs/source_type.png)


In [ ]:
# 2.1 截屏目标检测

from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Define current screenshot as source
source = "screen"

# Run inference on the source
results = model(source, stream=True)  # list of Results objects

results[0].show()

In [ ]:
# 2.2 摄像头目标检测

import cv2
from ultralytics import YOLO
import time

# Load the YOLO model
model = YOLO("yolo11n.pt")

# Open the default camera (0 is usually the built-in webcam)
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if not cap.isOpened():
    print("错误: 无法打开摄像头")
    exit()

print("摄像头已成功打开。按 'q' 键退出。")
window_name = "YOLO 摄像头目标检测"

try:
    # Loop through the camera frames
    while True:
        # Read a frame from the camera
        success, frame = cap.read()
        
        if success:
            # Run YOLO inference on the frame
            results = model(frame)
            
            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            
            # Display the annotated frame
            cv2.imshow(window_name, annotated_frame)
            
            key = cv2.waitKey(30) & 0xFF
            if key == ord("q") or cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
                break
        else:
            print("错误: 无法读取摄像头画面")
            break

finally:
    cv2.destroyAllWindows()
    cap.release()
    time.sleep(3)
    print("摄像头已释放，窗口已关闭。")

### 3. 推理参数

![image](docs/model_predict.png)

In [ ]:
# 3.1 推理参数分类

from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Run inference on 'bus.jpg' with arguments
results = model.predict(
                        # ---- 1. 基础输入与输出配置参数 -------------------------------------------------------------------
                        source = "https://ultralytics.com/images/bus.jpg", 
                        save = True,                # 控制是否保存推理结果
                        project = None,             # None  如果 save 已启用，则为保存预测输出的项目目录的名称。
                        name = None,                # None  预测运行的名称。用于在项目文件夹中创建一个子目录，如果 save 已启用，则为保存预测输出的项目目录的名称。
                        verbose = True,             # True  控制是否在终端中显示详细的推理日志，从而提供有关预测过程的实时反馈。
                                                                
                        # ---- 2. 图像 / 视频预处理参数 -------------------------------------------------------------------
                        imgsz = 320,                # 640   定义推理的图像大小。适当的大小调整可以提高检测 准确性 和处理速度。
                        batch = 1,                  # 1     指定推理的批处理大小（仅在源为以下情况时有效： 一个目录、视频文件，或 .txt 文件)。
                                                    #       更大的批处理大小可以提供更高的吞吐量，从而缩短推理所需的总时间。
                        vid_stride = 1,	            # 1     视频输入的帧步长。允许跳过视频中的帧，以加快处理速度，但会降低时间分辨率。值为 1 时处理每一帧，值越高跳过的帧越多。
                        stream_buffer = False,      # False 确定是否为视频流排队传入帧。

                        # ---- 3. 模型推理效率优化参数 -------------------------------------------------------------------
                        half = False,               # False 启用半精度 (FP16) 推理，这可以加快在支持的 GPU 上的模型推理速度，同时对准确性的影响极小。
                        device = None,              # None  指定用于推理的设备（例如， cpu, cuda:0 或 0）。允许用户在 CPU、特定 GPU 或其他计算设备之间进行选择，以执行模型。
                        stream = False,             # False 通过返回 Results 对象的生成器而不是一次将所有帧加载到内存中，从而为长视频或大量图像启用内存高效处理。
                        compile = False,            # False 启用 PyTorch 2.x torch.compile 使用以下方式进行图形编译 backend='inductor'。
                                                    #       接受   True → "default", 
                                                    #               False → 禁用，或字符串模式，
                                                    #       例如 "default", "reduce-overhead", "max-autotune-no-cudagraphs"。
                                                    #       如果不支持，则会发出警告并回退到 Eager 模式.

                        # ---- 4. 检测结果筛选与后处理参数 -------------------------------------------------------------------
                        conf = 0.5,                 # 0.25  设置检测的最小置信度阈值。 将忽略置信度低于此阈值的检测到的对象
                        iou = 0.7,                  # 0.7   设置非极大值抑制 (NMS) 的 IoU 阈值
                        max_det = 300,              # 300   每张图像允许的最大 detect 数量。限制模型在单次推理中可以 detect 的对象总数，防止在密集场景中产生过多输出。
                                                    #       如果 False，旧帧会被丢弃以适应新帧（针对实时应用进行了优化）。
                                                    #       如果 True，在缓冲区中对新帧进行排队，确保不跳过任何帧，但如果推理 FPS 低于流 FPS，则会导致延迟。
                        agnostic_nms = False,       # False 启用类别无关的非极大值抑制 (NMS)，它合并不同类别的重叠框。在类别重叠常见的多类别 detect 场景中非常有用。
                        classes = None,             # None  将预测结果筛选到一组类别 ID。只会返回属于指定类别的检测结果。这对于专注于多类别检测任务中的相关对象非常有用。
                        rect = True,                # True  (专属检测任务) 为矩形检测模式

                        # ---- 5. 高级功能与可视化参数 -------------------------------------------------------------------
                        visualize = False,          # False 激活推理期间模型特征的可视化，从而深入了解模型正在“看到”的内容。这对于调试和模型解释非常有用。
                        augment = False,            # False 启用测试时增强 (TTA) 进行预测，可能会提高检测的鲁棒性，但会降低推理速度。
                        retina_masks = None,        # False (专属分割任务) 返回高分辨率分割掩码。返回的掩码（masks.data）
                                                    #       如果启用，将与原始图像大小匹配。
                                                    #       如果禁用，它们将具有推理期间使用的图像大小。
                        embed = None,               # None  指定从中提取特征向量或 embeddings 的层。对于诸如聚类或相似性搜索之类的下游任务非常有用。

                        )

for result in results:
    result.show()